In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout
import tensorflow.keras.layers as layers
from spektral.layers import GCNConv, GlobalSumPool
import spektral.data as data
from spektral.data import Dataset
import tensorflow as tf
import numpy as np 
from tensorflow import keras
from tensorflow.keras import layers
from framasToGraph import FramsTransformer
from framsfiles import reader as framsreader
import spektral
import os
import fnmatch
from GraphDataset import GraphDataset
from autoencoder import Encoder, DecoderX, DecoderA, VGAE
import matplotlib.pyplot as plt

PATH_FRAMS='/home/adam/Framsticks/Framsticks50rc19'
PATH_DATA ="graphs/gen/"


Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data

Graph(n_nodes=30, n_node_features=3, n_edge_features=None, n_labels=None)


In [2]:
BATCH_SIZE = 256
latent_dim = 10
n_hidden = 64
ADJ_SIZE = 15
NUM_FEATURES = 3
epochs = 200
steps = 1000


myData = GraphDataset(PATH_FRAMS, PATH_DATA,size_of_adj=ADJ_SIZE)
# dataset.apply(normalize_one())
loader = data.BatchLoader(myData, batch_size=BATCH_SIZE)

myData

Using Framsticks version: 5.0rc19.1
Home (writable) dir     : /home/adam/Framsticks/Framsticks50rc19/data
Resources dir           : /home/adam/Framsticks/Framsticks50rc19/data



GraphDataset(n_graphs=353727)

In [3]:
encoder = Encoder(latent_dim=latent_dim,n_hidden=n_hidden) 
decoderA = DecoderA(ADJ_SIZE,latent_dim)
decoderX = DecoderX(latent_dim,ADJ_SIZE,NUM_FEATURES)

inputsX = tf.keras.Input(shape=(ADJ_SIZE,NUM_FEATURES))
inputsA = tf.keras.Input(shape=(ADJ_SIZE,ADJ_SIZE))
inp= [inputsX,inputsA]

autoencoder = VGAE(encoder,decoderA,decoderX)

opt = keras.optimizers.Adam(learning_rate=0.0001)
autoencoder.compile(optimizer=opt)
autoencoder._set_inputs(inp)

autoencoder.load_weights("models/Example/model")

losses_all = []


In [5]:
num_of_examples = 7

x,a = next(loader)

           
z_mean, z_log_var, z = autoencoder.encoder([tf.convert_to_tensor(x),tf.convert_to_tensor(a)])
dec_a = autoencoder.decoderA(z)

dec_x = autoencoder.decoderX([z,dec_a])

for i in range(num_of_examples):
    np.set_printoptions(precision=0,suppress=True)

    print("TRUE A")
    print(a[i])
    print("RECONSTRUCTED A")
    print(dec_a[i])
    np.set_printoptions(precision=2,suppress=True)

    print("TRUE X")
    print(x[i])
    print("RECONSTRUCTED X")
    print(dec_x[i])
    print("\n\n\n\n")

TRUE A
[[1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
RECONSTRUCTED A
tf.Tensor(
[[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 